### project stuff - hs

In [74]:
# importing libraries etc.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

<hr style="border: 2px solid #95e89e">

### I. filtering data to selected observation (top 20%)

#### what happens in these next 2 blocks of codes:

- importing datasets
- limiting the store dataset, `Rossmann_store.csv` to just the top 20% of stores in terms of sales
    - done by: calc avg sales for each store, then take top 20% of stores in terms of sales
- similarly, we wanna limit the other dataset we will use, `Rossmann_train.csv` so that it only gives us details of the top 20% sales stores
    - done by: filtering by the Store IDs based on those present in our t20 store df obtained earlier


<i>use this:</i>

In [75]:
# importing datasets
rossmann_store = pd.read_csv("Rossmann_store.csv")
rossmann_sales = pd.read_csv("Rossmann_train.csv") # from here on the train dataset will be referred to as the sales dataset

<i>followed by this:</i>

In [76]:
# creating a function to calculate the average sales for a store
def calc_avg_sales(store_id: int)-> float:
    """This function takes a store ID and returns the average sales value for that store."""
    a_store = rossmann_sales.loc[ (rossmann_sales['Store'] == store_id) ]
    avg_sales = a_store['Sales'].mean()
    return avg_sales.round(2)

# calculating and recording the average sales for each store
store_avg_sales = []
for i in range(1,1116):
    store_avg_sales.append(calc_avg_sales(i))

# adding the average sales data for each store into the store dataset
rossmann_store_w_avg = pd.concat([rossmann_store, pd.DataFrame({'AverageSales' : store_avg_sales})], axis=1)

# taking the top 20 percent of stores in terms of sales
rossmann_store_t20 = \
rossmann_store_w_avg.loc[ (rossmann_store_w_avg['AverageSales'] >= rossmann_store_w_avg['AverageSales'].quantile(0.8)) ]

# filtering the sales dataset
rossmann_sales_t20 = rossmann_sales.loc[ rossmann_sales['Store'].isin(rossmann_store_t20['Store']) ]

<hr style="border: 2px solid #73deb0">

### II. merging datasets & imputation of missing values for Promo2-related columns

#### what happens in this next block of code (done with method chaining):
- merged datasets
- convert `Date` dtype to datetime
- to reduce confusion, renamed `PromoInterval` to `Promo2Interval` since it's referring to Promo2

- filled in missing values for `Promo2SinceWeek`, `Promo2SinceYear`, and `Promo2Interval`
    - `Promo2SinceWeek`, `Promo2SinceYear` : replaced NaN with <b>0</b> if `Promo2` == 0
    - `Promo2Interval` : replaced NaN with <b>NA</b> (= Not Applicable) if `Promo2` == 0

- created a new column with category codes: `Promo2IntervalCat`
    - category <b>0</b> is <b>NA</b>
    - category <b>1</b> is <b>Jan,Apr,Jul,Oct</b>
    - category <b>2</b> is <b>Feb,May,Aug,Nov</b>
    - category <b>3</b> is <b>Mar,Jun,Sept,Dec</b>


<i>continue from previous section, now use this:</i>

In [122]:
rossmann = (
    pd.merge(rossmann_store_t20, rossmann_sales_t20, on='Store', how='right')   # merging the filtered datasets
    .assign(Date = lambda df: pd.to_datetime(df['Date']))                       # converting Date dtype from to datetime
    .rename(columns={'PromoInterval':'Promo2Interval'})                           # renaming PromoInterval to Promo2Interval
    .fillna({'Promo2SinceWeek':'0','Promo2SinceYear':'0','Promo2Interval':'NA'})  # filling in some missing values
    
    # creating a new column to categorize Promo2Interval
    .assign(Promo2IntervalCat = lambda df: df['Promo2Interval']
            .astype('category')
           )
    
    # changing categories to categorical codes for the new column
    .assign(Promo2IntervalCat = lambda df: pd.Categorical(df['Promo2IntervalCat'], categories= \
                                                          ['NA','Jan,Apr,Jul,Oct','Feb,May,Aug,Nov','Mar,Jun,Sept,Dec'])
            .codes
           )
    
    .set_index('Date')   # setting the Date as our index
)

### now can just use the database "rossmann" for further data pre-processing !

<hr style="border: 2px solid #73ded3">

<i>viewing a sample of the data (no code here; just for yalls to check how it looks like now):</i>

In [123]:
rossmann.sample(10)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Promo2Interval,AverageSales,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Promo2IntervalCat
Date,,,,,,,,,,,,,,,,,,,
2015-05-11,434,a,a,13020.0,8.0,2003.0,1,40.0,2014.0,"Jan,Apr,Jul,Oct",8127.70,1,8083,629,1,0,0,0,1
2014-03-16,826,a,c,7980.0,6.0,2005.0,0,0,0,NA,7281.54,7,0,0,0,0,0,0,0
2015-04-05,1018,c,c,140.0,9.0,2012.0,0,0,0,NA,9752.87,7,0,0,0,0,0,0,0
2015-05-14,817,a,a,140.0,3.0,2006.0,0,0,0,NA,18108.14,4,0,0,0,0,a,0,0
2014-02-12,862,a,c,2840.0,3.0,2010.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",7128.52,3,7105,1072,1,0,0,0,1
2013-11-25,106,a,a,1390.0,8.0,2013.0,0,0,0,NA,7131.99,1,9719,914,1,0,0,0,0
2015-04-08,672,c,a,240.0,9.0,2002.0,0,0,0,NA,7317.35,3,7661,1265,1,0,0,1,0
2013-12-04,1112,c,c,1880.0,4.0,2006.0,0,0,0,NA,8465.28,3,12382,921,1,1,0,0,0
2015-05-28,1089,d,a,5220.0,5.0,2009.0,0,0,0,NA,8402.58,4,9230,895,1,0,0,0,0
